### 유튜브 랭킹 크롤링
* https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=2
* 순위, 카테고리, 채널, 구독자수, 조회수, 비디오수
* 셀레늄으로 해당페이지를 열고 + bs4를 사용해서 크롤링
* 10페이지

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from tqdm.notebook import tqdm
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
#import re  # 정규표현식

In [2]:
driver = webdriver.Chrome()
url1 = 'https://youtube-rank.com/'
page = [str(i+1) for i in range(10)]   
page


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [20]:
line = []

rank, ctgr, chnl = '---', '---', '---'
subs, stro, vdeo = '---', '---', '---'
view = '---'

# 순위, 카테고리, 채널, 구독자수, view수, 조회수, 비디오수
for i  in range(len(page)):
#for i  in range(1):    # page loop

    url = url1 + f'board/bbs/board.php?bo_table=youtube&page={i}'
    #https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&wr_id=4979&page=9
    driver.get(url)
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    songs = soup.select('tbody tr.aos-init')
    #print(len(songs))


    for j in range(len(songs)): # songs loop in 1 page
    #for j in [1]: # songs loop in 1 page
        # rank
        rank = songs[j].select_one('td.rank').text.strip()
        # 카테고리
        ctgr = songs[j].select_one('h1 p.category').text.strip()

        # 채널
        #chnl_link = songs[j].select_one('div.info_img a')['href'].strip()
        #driver.get(chnl_link)
        #time.sleep(1)
        #html_sub = driver.page_source
        #soup_sub = BeautifulSoup(html_sub, 'html.parser')

        try:
            chnl = songs[j].select_one('h1 a').text.strip()
            #chnl = soup_sub.select_one('ul.subInfo li strong').text.strip()
        except:
            chnl = songs[j].select_one('h1 a')
            #chnl = soup_sub.select_one('ul.subInfo li strong').text.strip()
        # 구독자수
        subs = songs[j].select_one('td.subscriber_cnt').text.strip()
        # 조회수
        stro = songs[j].select_one('td.hit strong').text.strip()
        # 비디오수
        vdeo = songs[j].select_one('td.video_cnt').text.strip()
        # view 수
        view = songs[j].select_one('td.view_cnt').text.strip()
        view = int(view.replace('억',"").replace('만',""))*10000
        view = f'{int(view):3,}'

        line.append([rank, ctgr, chnl, subs, view, stro, vdeo])
#
df = pd.DataFrame(line, columns=['순위', '카테고리', '채널', '구독자수', '뷰수','조회수', '비디오수'])
df






,순위,카테고리,채널,구독자수,뷰수,조회수,비디오수
0,1,[음악/댄스/가수],BLACKPINK,8680만,"30,340,160,000","25,954",505개
1,2,[음악/댄스/가수],BANGTANTV,7430만,"20,078,950,000","10,841","2,174개"
2,3,[음악/댄스/가수],HYBE LABELS,7010만,"27,213,060,000","12,964","1,142개"
3,4,[음악/댄스/가수],SMTOWN,3160만,"26,746,800,000","7,755","4,108개"
4,5,[음악/댄스/가수],JYP Entertainment,2670만,"19,236,390,000","6,906","1,662개"
...,...,...,...,...,...,...,...
995,896,[미분류],교양만두,80만,"238,310,000",958,221개
996,897,[음악/댄스/가수],DKDKTV,79만,"205,390,000","2,915","1,405개"
997,899,[미분류],DAWN,79만,"65,160,000",847,43개
998,898,[게임],김성회의 G식백과,79만,"219,720,000","2,605",383개


In [21]:
df.to_csv('../daou/유튜브랭킹out', index=False)